# Создание признаков

## 1 Импорт библиотек

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import pickle

from preprocessing_utils import haversine_distance, extract_address_from_coords

## 2 Подготовка данных

Набор данных после предобработки

In [2]:
#Набор данных после предобработки
df = pd.read_csv('houseprices_cleaned.csv')
#Рейтинг районов
district_ratings = pd.read_csv('district_ratings.csv')
#Координаты станций
metros = pd.read_csv('metro_coords.csv')

## 3 Район
Используя API Яндекс-карт получим район Москвы для каждого объявления (так как этот процесс занимает ~3 часа, я предварительно получил район, а затем перешел к созданию других признаков)

In [3]:
#df = df.apply(extract_address_from_coords, axis=1)
df.head()

,ID,Ссылка,Цена,Дата,Адрес,Этаж,Количество_комнат,Балкон_или_лоджия,Тип_комнат,Общая_площадь,...,В_доме_мусоропровод,В_доме_консьерж,В_доме_газ,Двор_закрытая_территория,Двор_спортивная_площадка,Двор_детская_площадка,Парковка,Количество_лифтов,Цена_квм,Район
0,2717185132,https://avito.ru//moskva/kvartiry/1-k._kvartir...,9000000,2022-12-29,"Москва, Сколковское шоссе, 30",2.0,1,балкон,NaN,37.7,...,1.0,0.0,0.0,NaN,NaN,NaN,0.0,2,238726.790451,Можайский
1,2742183245,https://avito.ru//moskva/kvartiry/1-k._kvartir...,10500000,2022-12-29,"Москва, Озёрная улица, 19к2",3.0,1,лоджия,NaN,35.0,...,1.0,0.0,0.0,NaN,NaN,NaN,0.0,2,300000.000000,Очаково-Матвеевское
2,2550551003,https://avito.ru//moskva/kvartiry/2-k._kvartir...,18500000,2022-12-29,"Москва, Варшавское шоссе, 94",9.0,2,NaN,изолированные,70.0,...,1.0,1.0,0.0,NaN,NaN,NaN,1.0,2,264285.714286,Нагорный
3,2581869272,https://avito.ru//moskva/kvartiry/2-k._kvartir...,13998999,2022-12-29,"Москва, улица Академика Арцимовича, 8",4.0,2,лоджия,изолированные,52.0,...,1.0,0.0,0.0,NaN,NaN,NaN,1.0,2,269211.519231,Коньково
4,2673088174,https://avito.ru//moskva/kvartiry/apartamenty-...,3990000,2022-12-29,"Москва, ул. Бутлерова, 24",1.0,студия,NaN,NaN,19.1,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,208900.523560,Коньково


## 4 Рейтинг района
Добавим данные о рейтингах районов Москвы

In [4]:
district_ratings.head()

,Район,Общий_рейтинг,Экология,Чистота,ЖКХ,Соседи,Условия_для_детей,Спорт_и_отдых,Магазины,Транспорт,Безопасность,Стоимость_жизни
0,Куркино,1,1,1,1,1,1,1,4,1,1,1
1,Строгино,2,3,5,3,5,3,2,11,26,11,4
2,Митино,3,9,3,5,13,4,3,6,8,13,16
3,Марфино,4,2,2,9,11,27,85,68,39,3,2
4,Тропарёво-Никулино,5,6,27,7,4,41,8,25,10,14,7


In [5]:
# удалим слово район из набора данных
df['Район'] = df['Район'].str.replace('район', '').str.strip()
# суммарный рейтинг - сумма всех рейтингов для района
district_ratings['Суммарный рейтинг'] = district_ratings.iloc[:,
                                                              1:].sum(axis=1)
# объединям рейтинг районов и набор данных
df = pd.merge(df, district_ratings, on='Район', how='left')
df.sample(5)

,ID,Ссылка,Цена,Дата,Адрес,Этаж,Количество_комнат,Балкон_или_лоджия,Тип_комнат,Общая_площадь,...,Чистота,ЖКХ,Соседи,Условия_для_детей,Спорт_и_отдых,Магазины,Транспорт,Безопасность,Стоимость_жизни,Суммарный рейтинг
19055,2686741386,https://avito.ru//moskva/kvartiry/3-k._kvartir...,24000000,2023-01-10,"Москва, Каширское ш., 74А",20.0,3,лоджия,изолированные,81.0,...,20.0,39.0,45.0,48.0,83.0,104.0,7.0,25.0,84.0,569.0
22264,2761622641,https://avito.ru//moskva/kvartiry/3-k._kvartir...,19350000,2023-01-14,"Москва, Бескудниковский б-р, 13",3.0,3,балкон,изолированные,77.0,...,69.0,82.0,100.0,61.0,93.0,88.0,101.0,99.0,106.0,973.0
3596,2666776214,https://avito.ru//moskva/kvartiry/1-k._kvartir...,11000000,2022-12-31,"Москва, б-р Яна Райниса, 45к1",5.0,1,NaN,NaN,37.6,...,38.0,63.0,95.0,63.0,19.0,56.0,65.0,39.0,67.0,557.0
21529,2764376605,https://avito.ru//moskva/kvartiry/5-k._kvartir...,75000000,2023-01-14,"Москва, ул. Пресненский Вал, 14к6",12.0,5,балкон,изолированные,160.0,...,45.0,67.0,16.0,85.0,52.0,109.0,40.0,10.0,79.0,658.0
3090,2557360286,https://avito.ru//moskva/kvartiry/kvartira-stu...,8850000,2022-12-31,"Москва, Нижегородская улица, 74",9.0,студия,лоджия,NaN,25.5,...,95.0,97.0,110.0,112.0,108.0,100.0,86.0,98.0,38.0,1067.0


## 5 Признаки на основе группировки
Для каждого значения количества комнат вычислим минимальную, максимальную, среднюю площадь. И добавим признаки - отношение общей площади квартиры к минимальной/максимальной/средней для данного количества комнат, а также разность между ними

In [6]:
df['Общая_площадь_отн_мин'] = (
    df['Общая_площадь'] /
    df.groupby('Количество_комнат')['Общая_площадь'].transform('min'))

df['Общая_площадь_минус_мин'] = (
    df['Общая_площадь'] -
    df.groupby('Количество_комнат')['Общая_площадь'].transform('min'))

df['Общая_площадь_отн_макс'] = (
    df['Общая_площадь'] /
    df.groupby('Количество_комнат')['Общая_площадь'].transform('max'))

df['Общая_площадь_минус_макс'] = (
    df['Общая_площадь'] -
    df.groupby('Количество_комнат')['Общая_площадь'].transform('max'))

df['Общая_площадь_отн_сред'] = (
    df['Общая_площадь'] /
    df.groupby('Количество_комнат')['Общая_площадь'].transform('mean'))

df['Общая_площадь_минус_сред'] = (
    df['Общая_площадь'] -
    df.groupby('Количество_комнат')['Общая_площадь'].transform('mean'))

df.head()

,ID,Ссылка,Цена,Дата,Адрес,Этаж,Количество_комнат,Балкон_или_лоджия,Тип_комнат,Общая_площадь,...,Транспорт,Безопасность,Стоимость_жизни,Суммарный рейтинг,Общая_площадь_отн_мин,Общая_площадь_минус_мин,Общая_площадь_отн_макс,Общая_площадь_минус_макс,Общая_площадь_отн_сред,Общая_площадь_минус_сред
0,2717185132,https://avito.ru//moskva/kvartiry/1-k._kvartir...,9000000,2022-12-29,"Москва, Сколковское шоссе, 30",2.0,1,балкон,NaN,37.7,...,110.0,83.0,103.0,991.0,2.513333,22.7,0.397260,-57.2,1.009300,0.347397
1,2742183245,https://avito.ru//moskva/kvartiry/1-k._kvartir...,10500000,2022-12-29,"Москва, Озёрная улица, 19к2",3.0,1,лоджия,NaN,35.0,...,107.0,77.0,94.0,943.0,2.333333,20.0,0.368809,-59.9,0.937016,-2.352603
2,2550551003,https://avito.ru//moskva/kvartiry/2-k._kvartir...,18500000,2022-12-29,"Москва, Варшавское шоссе, 94",9.0,2,NaN,изолированные,70.0,...,55.0,43.0,107.0,767.0,2.692308,44.0,0.397727,-106.0,1.259395,14.417735
3,2581869272,https://avito.ru//moskva/kvartiry/2-k._kvartir...,13998999,2022-12-29,"Москва, улица Академика Арцимовича, 8",4.0,2,лоджия,изолированные,52.0,...,38.0,51.0,54.0,369.0,2.000000,26.0,0.295455,-124.0,0.935550,-3.582265
4,2673088174,https://avito.ru//moskva/kvartiry/apartamenty-...,3990000,2022-12-29,"Москва, ул. Бутлерова, 24",1.0,студия,NaN,NaN,19.1,...,38.0,51.0,54.0,369.0,1.910000,9.1,0.303175,-43.9,0.987438,-0.242979


## 6 Расстояние до станций кольцевой линии

In [7]:
line_5 = [
    'Парк культуры', 'Октябрьская', 'Добрынинская', 'Павелецкая', 'Таганская',
    'Красные ворота', 'Проспект Мира', 'Курская', 'Белорусская',
    'Новослободская', 'Киевская', 'Краснопресненская'
]

In [8]:
for station in line_5:
    df.loc[:,
           f'Расстояние_до_{station.replace(" ","_")}'] = (haversine_distance(
               df['Широта'].values, df['Долгота'].values,
               metros.loc[metros['Станция'] == station, 'Широта'].item(),
               metros.loc[metros['Станция'] == station, 'Долгота'].item()))

In [9]:
df.head()

,ID,Ссылка,Цена,Дата,Адрес,Этаж,Количество_комнат,Балкон_или_лоджия,Тип_комнат,Общая_площадь,...,Расстояние_до_Добрынинская,Расстояние_до_Павелецкая,Расстояние_до_Таганская,Расстояние_до_Красные_ворота,Расстояние_до_Проспект_Мира,Расстояние_до_Курская,Расстояние_до_Белорусская,Расстояние_до_Новослободская,Расстояние_до_Киевская,Расстояние_до_Краснопресненская
0,2717185132,https://avito.ru//moskva/kvartiry/1-k._kvartir...,9000000,2022-12-29,"Москва, Сколковское шоссе, 30",2.0,1,балкон,NaN,37.7,...,13.932702,14.944434,16.123870,16.719713,16.479974,16.977819,13.609754,14.740037,10.810491,12.353293
1,2742183245,https://avito.ru//moskva/kvartiry/1-k._kvartir...,10500000,2022-12-29,"Москва, Озёрная улица, 19к2",3.0,1,лоджия,NaN,35.0,...,11.869950,12.801297,14.284220,15.677392,15.887513,15.565543,13.642113,14.551491,9.953380,11.938904
2,2550551003,https://avito.ru//moskva/kvartiry/2-k._kvartir...,18500000,2022-12-29,"Москва, Варшавское шоссе, 94",9.0,2,NaN,изолированные,70.0,...,9.097904,9.263357,10.805857,13.589382,14.749962,12.638727,14.661188,14.762669,11.198761,12.850535
3,2581869272,https://avito.ru//moskva/kvartiry/2-k._kvartir...,13998999,2022-12-29,"Москва, улица Академика Арцимовича, 8",4.0,2,лоджия,изолированные,52.0,...,11.793822,12.484369,14.153216,16.304568,16.964052,15.796682,15.539091,16.149532,11.560462,13.634330
4,2673088174,https://avito.ru//moskva/kvartiry/apartamenty-...,3990000,2022-12-29,"Москва, ул. Бутлерова, 24",1.0,студия,NaN,NaN,19.1,...,10.461154,11.081071,12.759555,15.051738,15.815691,14.458051,14.651203,15.159871,10.696308,12.729572


## 7 Кластеризация
Кластеризуем объявления по координатам и по значениям общей площади

In [10]:
kmeans_square = KMeans(n_clusters=10, random_state=42,n_init='auto')
kmeans_square.fit(df['Общая_площадь'].values.reshape(-1, 1))
# 125 - количество районов Москвы
kmeans_geo = KMeans(n_clusters=125, random_state=42,n_init='auto')
kmeans_geo.fit(df[['Широта', 'Долгота']])

df['Общая_площадь_кластер'] = kmeans_square.labels_
df['Координаты_кластер'] = kmeans_geo.labels_

In [11]:
# сохранение
with open("geo_cluster.pkl", "wb") as f:
    pickle.dump(kmeans_geo, f)
with open("square_cluster.pkl", "wb") as f:
    pickle.dump(kmeans_square, f)

In [26]:
df=pd.read_csv('houseprices_final.csv')

In [28]:
df.loc[df['Тип_дома']=='монолитнокирпичный','Тип_дома']='монолитно-кирпичный'

In [27]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [31]:
cat_features=['Район','Количество_комнат','Ремонт','Тип_дома']

encoder=OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
df[cat_features]=encoder.fit_transform(df[cat_features])

In [30]:
df['Тип_дома'].value_counts()

панельный              7657
монолитный             7100
кирпичный              5379
монолитно-кирпичный    1326
блочный                 974
Name: Тип_дома, dtype: int64

In [32]:
with open("categorical_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

In [24]:
import sklearn

In [25]:
sklearn.__version__

'1.2.1'

## 8 Сохранение

In [33]:
#df = df.sort_values(by='Дата').reset_index(drop=True)
df.to_csv('houseprices_final.csv', index=None)